##Скачиваем датасет

In [1]:
!wget -P /content  https://raw.githubusercontent.com/julianzim/suppi_trial_task/main/trial_task.json

--2023-09-19 09:49:34--  https://raw.githubusercontent.com/julianzim/suppi_trial_task/main/trial_task.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40594 (40K) [text/plain]
Saving to: ‘/content/trial_task.json’

trial_task.json     100%[===================>]  39.64K  --.-KB/s    in 0.02s   

2023-09-19 09:49:35 (2.56 MB/s) - ‘/content/trial_task.json’ saved [40594/40594]



##Импорты

In [2]:
import pandas as pd
import numpy as np

##Читаем данные

In [3]:
df = pd.read_json('/content/trial_task.json')
df.head()

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


##Задание 1
Найти тариф стоимости доставки для каждого склада

In [4]:
df['highway_price'] = df['highway_cost'] / df.products.apply(lambda x: sum([line['quantity'] for line in x]))
df.head()

,order_id,warehouse_name,highway_cost,products,highway_price
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':...",-10.0
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,...",-15.0
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,...",-25.0
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,...",-10.0
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric...",-25.0


In [5]:
df.products[4]

[{'product': 'автограф Стаса Барецкого', 'price': 600, 'quantity': 1},
 {'product': 'статуэтка Ленина', 'price': 200, 'quantity': 1},
 {'product': 'плюмбус', 'price': 250, 'quantity': 1}]

In [6]:
highway_prices = df[['warehouse_name', 'highway_price']].drop_duplicates()
highway_prices

,warehouse_name,highway_price
0,Мордор,-10.0
1,хутор близ Диканьки,-15.0
2,отель Лето,-25.0
5,остров невезения,-5.0
6,гиперборея,-20.0


##Задание 2
Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами `'product', 'quantity', 'income', 'expenses', 'profit'`)

In [7]:
keys = list(df.products[0][0].keys())
keys

['product', 'price', 'quantity']

In [8]:
exploded_df = df.explode('products', ignore_index=True)
exploded_df[keys] = exploded_df.products.apply(lambda x: pd.Series([x.get(keys[0]), x.get(keys[1]), x.get(keys[2])]))
exploded_df.drop(['products'], axis=1, inplace=True)
exploded_df.head()

,order_id,warehouse_name,highway_cost,highway_price,product,price,quantity
0,11973,Мордор,-70,-10.0,ломтик июльского неба,450,1
1,11973,Мордор,-70,-10.0,билет в Израиль,1000,3
2,11973,Мордор,-70,-10.0,статуэтка Ленина,200,3
3,62239,хутор близ Диканьки,-15,-15.0,билет в Израиль,1000,1
4,85794,отель Лето,-50,-25.0,зеленая пластинка,10,2


In [9]:
summary = exploded_df[['product', 'quantity']].groupby('product').sum().reset_index()

income = exploded_df.groupby('product').apply(
    lambda x: sum(x['price'] * x['quantity'])).reset_index()
expenses = exploded_df.groupby('product').apply(
    lambda x: sum(x['highway_price'] * x['quantity'])).reset_index()

summary = summary.merge(income, on='product')
summary = summary.merge(expenses, on='product')
summary.columns = ['product', 'quantity', 'income', 'expenses']
summary['profit'] = summary.income + summary.expenses
summary

,product,quantity,income,expenses,profit
0,автограф Стаса Барецкого,48,28800,-820.0,27980.0
1,билет в Израиль,58,58000,-1050.0,56950.0
2,зеленая пластинка,61,610,-920.0,-310.0
3,ломтик июльского неба,42,18900,-670.0,18230.0
4,плюмбус,65,16250,-940.0,15310.0
5,подписка на suppi-блог,33,4950,-545.0,4405.0
6,статуэтка Ленина,68,13600,-935.0,12665.0


##Задание 3
Составить табличку со столбцами `'order_id'` (id заказа) и `'order_profit'` (прибыль полученная с заказа). А также вывести среднюю прибыль заказов

In [10]:
exploded_df.query('order_id == order_id[0]')   #глянуть на один из заказов

,order_id,warehouse_name,highway_cost,highway_price,product,price,quantity
0,11973,Мордор,-70,-10.0,ломтик июльского неба,450,1
1,11973,Мордор,-70,-10.0,билет в Израиль,1000,3
2,11973,Мордор,-70,-10.0,статуэтка Ленина,200,3


In [11]:
def profit(df):
    '''Считает прибыль по заказу, которая равна разности дохода и
    расходов по заказу
    '''
    income = df['price'] * df['quantity']
    expenses = df['highway_price'] * df['quantity']
    return sum(income) + sum(expenses)

In [12]:
order_report = exploded_df.groupby('order_id').apply(profit).reset_index()
order_report.columns = ['order_id', 'order_profit']
order_report.head()

,order_id,order_profit
0,124,705.0
1,1391,490.0
2,2091,1300.0
3,2108,200.0
4,2558,355.0


In [13]:
average_order_profit = order_report.order_profit.mean()
average_order_profit

1352.3

##Задание 4
Составить табличку типа `'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse'` (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

In [31]:
warehouse_product_count = exploded_df.groupby(['warehouse_name', 'product']).apply(lambda x: x['quantity'].sum()).reset_index()
warehouse_product_count.rename(columns={0: 'quantity'}, inplace=True)
warehouse_product_count.head()

,warehouse_name,product,quantity
0,Мордор,автограф Стаса Барецкого,4
1,Мордор,билет в Израиль,9
2,Мордор,зеленая пластинка,11
3,Мордор,ломтик июльского неба,3
4,Мордор,плюмбус,6


In [32]:
warehouse_product_profit = exploded_df.groupby(['warehouse_name', 'product']).apply(profit).reset_index()
warehouse_product_profit.rename(columns={0: 'profit'}, inplace=True)

warehouse_product_profit = warehouse_product_count.merge(warehouse_product_profit, on=['warehouse_name', 'product'])
warehouse_product_profit.head()

,warehouse_name,product,quantity,profit
0,Мордор,автограф Стаса Барецкого,4,2360.0
1,Мордор,билет в Израиль,9,8910.0
2,Мордор,зеленая пластинка,11,0.0
3,Мордор,ломтик июльского неба,3,1320.0
4,Мордор,плюмбус,6,1440.0


In [33]:
def percent_profit(df):
    return round(df['profit'] / sum(df['profit']) * 100, 2)

In [34]:
percents = warehouse_product_profit.groupby(['warehouse_name']).apply(percent_profit).reset_index()['profit']
warehouse_product_profit['percent_profit_product_of_warehouse'] = percents
warehouse_product_profit.head(10)

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,автограф Стаса Барецкого,4,2360.0,13.84
1,Мордор,билет в Израиль,9,8910.0,52.26
2,Мордор,зеленая пластинка,11,0.0,0.00
3,Мордор,ломтик июльского неба,3,1320.0,7.74
4,Мордор,плюмбус,6,1440.0,8.45
5,Мордор,подписка на suppi-блог,8,1120.0,6.57
6,Мордор,статуэтка Ленина,10,1900.0,11.14
7,гиперборея,автограф Стаса Барецкого,12,6960.0,18.18
8,гиперборея,билет в Израиль,21,20580.0,53.76
9,гиперборея,зеленая пластинка,10,-100.0,-0.26


##Задание 5
Взять предыдущую табличку и отсортировать `'percent_profit_product_of_warehouse'` по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться `'accumulated_percent_profit_product_of_warehouse'`. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца `'percent_profit_product_of_warehouse'`.

In [55]:
warehouse_product_profit_sorted = warehouse_product_profit.groupby('warehouse_name').apply(
    lambda x: x.sort_values(['percent_profit_product_of_warehouse'], ascending=False)
    ).reset_index(drop=True)
warehouse_product_profit_sorted.head(10)

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,билет в Израиль,9,8910.0,52.26
1,Мордор,автограф Стаса Барецкого,4,2360.0,13.84
2,Мордор,статуэтка Ленина,10,1900.0,11.14
3,Мордор,плюмбус,6,1440.0,8.45
4,Мордор,ломтик июльского неба,3,1320.0,7.74
5,Мордор,подписка на suppi-блог,8,1120.0,6.57
6,Мордор,зеленая пластинка,11,0.0,0.00
7,гиперборея,билет в Израиль,21,20580.0,53.76
8,гиперборея,автограф Стаса Барецкого,12,6960.0,18.18
9,гиперборея,ломтик июльского неба,13,5590.0,14.60


In [56]:
warehouse_product_profit_sorted['accumulated_percent_profit_product_of_warehouse'] =\
    warehouse_product_profit_sorted.groupby('warehouse_name').apply(
        lambda x: np.cumsum(x['percent_profit_product_of_warehouse'])
        ).reset_index(drop=True)
warehouse_product_profit_sorted.head(10)

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
0,Мордор,билет в Израиль,9,8910.0,52.26,52.26
1,Мордор,автограф Стаса Барецкого,4,2360.0,13.84,66.10
2,Мордор,статуэтка Ленина,10,1900.0,11.14,77.24
3,Мордор,плюмбус,6,1440.0,8.45,85.69
4,Мордор,ломтик июльского неба,3,1320.0,7.74,93.43
5,Мордор,подписка на suppi-блог,8,1120.0,6.57,100.00
6,Мордор,зеленая пластинка,11,0.0,0.00,100.00
7,гиперборея,билет в Израиль,21,20580.0,53.76,53.76
8,гиперборея,автограф Стаса Барецкого,12,6960.0,18.18,71.94
9,гиперборея,ломтик июльского неба,13,5590.0,14.60,86.54


##Задание 6
Присвоить A,B,C - категории на основании значения накопленного процента (`'accumulated_percent_profit_product_of_warehouse'`). Если значение накопленного процента меньше или равно 70, то категория A.
Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как `'category'`

In [57]:
def assign_category(number):
    if number <= 70:
        return 'A'
    elif 70 < number <= 90:
        return 'B'
    else:
        return 'C'

In [58]:
warehouse_product_profit_sorted['category'] =\
    warehouse_product_profit_sorted['accumulated_percent_profit_product_of_warehouse'].apply(assign_category)
warehouse_product_profit_sorted.head(10)

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
0,Мордор,билет в Израиль,9,8910.0,52.26,52.26,A
1,Мордор,автограф Стаса Барецкого,4,2360.0,13.84,66.10,A
2,Мордор,статуэтка Ленина,10,1900.0,11.14,77.24,B
3,Мордор,плюмбус,6,1440.0,8.45,85.69,B
4,Мордор,ломтик июльского неба,3,1320.0,7.74,93.43,C
5,Мордор,подписка на suppi-блог,8,1120.0,6.57,100.00,C
6,Мордор,зеленая пластинка,11,0.0,0.00,100.00,C
7,гиперборея,билет в Израиль,21,20580.0,53.76,53.76,A
8,гиперборея,автограф Стаса Барецкого,12,6960.0,18.18,71.94,B
9,гиперборея,ломтик июльского неба,13,5590.0,14.60,86.54,B
